In [1]:
import importlib
import sys
from tqdm import tqdm
import  os
import pickle
import glob
from datetime import datetime
import matplotlib.pyplot as plt
# Add the directories to the sys.path
dir_path = "C://Users//jony//Programming//Python//Anubis//anubis//" # insert your directory path
sys.path.append(dir_path + "Osiris//processing//python")
sys.path.append(dir_path + "tools")

import Analysis_tools as ATools
import Atlas_tools as AtTools
import proAnubis_Analysis_Tools
import Reconstruction_tools as RTools
import numpy as np
import mplhep as hep
import Timing_tools as TTools
import rawFileReader
import Visual_tools as VTools
hep.style.use([hep.style.ATLAS])

# Specify the directory
data_list = sorted([f for f in os.listdir("data//reset") if os.path.isfile(os.path.join("data//reset", f))], reverse=True) ##all files in diffT directory sorted from the newest to the oldest

file_path = list(map(lambda p: dir_path+"data//reset//"+p, data_list))[:1] #
print(file_path)

['C://Users//jony//Programming//Python//Anubis//anubis//data//reset//proAnubis_240810_0040.raw']


In [6]:

importlib.reload(AtTools)
import pickle

anubis_file = "data.pkl" #'C://Users//jony//Programming//Python//Anubis//anubis//data//proAnubis_240627_1720.raw'
analyser = AtTools.AtlasAnalyser()
analyser.getTDC5Data(anubis_file, trigger_channel=4,bcr_channel=2, fromPKL=True, amount_of_events=10000)
#c = analyser.correctionBCR()
#print(c/len(analyser.anubis_data))

Jump 11
Expecting 205953
Actual 263689
Difference: 57736
Mod Difference: 57736
Error time: 2024-06-27 17:23:08.002159
Jump 5
Expecting 570240
Actual 570240
Difference: 0
Mod Difference: 0
Error time: 2024-06-27 17:23:08.002159
Jump 3
Expecting 342144
Actual 342145
Difference: 1
Mod Difference: 1
Error time: 2024-06-27 17:23:08.005267
Jump 3
Expecting 342144
Actual 342142
Difference: -2
Mod Difference: 114046
Error time: 2024-06-27 17:23:08.005267
Jump 6
Expecting 684288
Actual 1026431
Difference: 342143
Mod Difference: 114047
Error time: 2024-06-27 17:23:08.008410
Jump 8
Expecting 912384
Actual 91903
Difference: -820481
Mod Difference: 91903
Error time: 2024-06-27 17:23:08.008410
Jump 3
Expecting 342144
Actual 342143
Difference: -1
Mod Difference: 114047
Error time: 2024-06-27 17:23:08.011656
Jump 3
Expecting 342144
Actual 570240
Difference: 228096
Mod Difference: 0
Error time: 2024-06-27 17:23:08.017896
Jump 2
Expecting 228096
Actual 342143
Difference: 114047
Mod Difference: 114047
Er

In [50]:
def get_corrected(max_offset):
    previous_hit_time = analyser.anubis_data[0].hitTime
    diffT = []
    for bcr in analyser.anubis_data[1:]: 
        delta = bcr.hitTime - previous_hit_time
        if delta < 0:
            delta += 1_048_575
        if delta>1_000_0000:
            #print("ERROR")
            #print(delta)
            #print(bcr.hitTime)
            #print(previous_hit_time)
            continue
        if abs(delta - 114_047) > max_offset:
            previous_hit_time = (previous_hit_time + 114_047) % 1_048_575
        else:
            previous_hit_time = bcr.hitTime
        diffT.append(delta)
    return diffT

In [84]:
find_std = []
total = 100_0000
with tqdm(total=total, desc=f"Processing {anubis_file}", unit='BCR') as pbar:
    for max_offset in range(0, total, 500):
        
        diffT = get_corrected(max_offset)    
        #plot historgram of the time difference between hits
        find_std.append(np.std(diffT))
        pbar.update(500)

Processing data.pkl: 100%|██████████| 1000000/1000000 [04:34<00:00, 3647.41BCR/s]


In [85]:
plt.plot(range(0, len(find_std)*500, 500), find_std, label="Our data")
plt.xlabel("Max offset")
plt.ylabel("Standard deviation")
plt.title("Standard deviation of the time difference between hits")
plt.plot(range(0, len(find_std)*500, 500),[3.02e5]*len(find_std), label="Uniform data")
plt.legend()
plt.show()

In [80]:
diffT = get_corrected(7_0_000)
num_bins = int(np.sqrt(len(diffT)))
# Generate histogram data
counts, bin_edges = np.histogram(diffT, bins=num_bins)

# Plot the step plot
plt.step(bin_edges[:-1], counts, where='mid', label='Step Plot')

# Optionally, you can also display the histogram as a bar plot for comparison
plt.hist(diffT, bins=num_bins, alpha=0.5, label='Histogram')

# Add labels and title
plt.yscale('log')
plt.ylabel('Frequency')
plt.title(f'Time difference between BCR')

# Show legend
plt.legend()

# Show the plot
plt.show()


###The precision over Time

In [45]:
diffT = get_corrected(70_000_000)
plt.plot(diffT[:1000])
plt.show()

In [47]:
importlib.reload(AtTools)
import pickle

anubis_file = "data.pkl" #'C://Users//jony//Programming//Python//Anubis//anubis//data//proAnubis_240627_1720.raw'
analyser = AtTools.AtlasAnalyser()
analyser.getTDC5Data(anubis_file, trigger_channel=4,bcr_channel=2, fromPKL=True, amount_of_events=1000)
#plt.plot([100_000*int(analyser.anubis_data[d].error) for d in range(1000)], label = "analyser")
#plt.plot(diffT[:1000], label="Our data")
plt.title("Time difference between BCR")
plt.xlabel("BCR")
plt.ylabel("Time difference")
plt.plot(analyser.tDiff[:1000], label="Time interval")
#analyser.correctionBCR()
#plt.plot([200_000*int(analyser.anubis_data[d].error) for d in range(1000)], label = "after")
plt.legend()
plt.show()

In [18]:
importlib.reload(AtTools)
import pickle

anubis_file = "data.pkl" #'C://Users//jony//Programming//Python//Anubis//anubis//data//proAnubis_240627_1720.raw'
analyser = AtTools.AtlasAnalyser()
analyser.getTDC5Data(anubis_file, trigger_channel=4,bcr_channel=2, fromPKL=True, amount_of_events=1000)

times = []
max_event = 100
for bcr in analyser.anubis_data[:max_event]:
    times.append(bcr.hitTime)

plt.plot(times, label = "original")
plt.plot([-100_000*int(analyser.anubis_data[d].error) for d in range(max_event)], label = "original flags")
#analyser.correctionBCR()
times = []

for bcr in analyser.anubis_data[:max_event]:
    times.append(bcr.hitTime)
plt.plot(times, label = "after")
plt.plot([-50_000*int(analyser.anubis_data[d].error) for d in range(max_event)], label = "after flags")
plt.legend()
plt.show()

In [15]:
problems = 0
for bcr in analyser.anubis_data:
    if bcr.error:
        problems += 1
print(problems/len(analyser.anubis_data))

0.1610196326789107


In [108]:
def plot_BCD(bcd_lists):
    labels = ["before", "after"]
    markers = ["o", "x"]
    for k, bcd_list in enumerate(bcd_lists):

        

SyntaxError: incomplete input (2722991618.py, line 6)

In [14]:
importlib.reload(AtTools)
anubis_file = "data.pkl" #'C://Users//jony//Programming//Python//Anubis//anubis//data//proAnubis_240627_1720.raw'
analyser = AtTools.AtlasAnalyser()
analyser.getTDC5Data(anubis_file, trigger_channel=4,bcr_channel=2, fromPKL=True, amount_of_events=1000)

max_event = 100
before = analyser.anubis_data[:max_event].copy()
#analyser.printBCR(10)
#analyser.correctionBCR()
#after = analyser.anubis_data[1:200].copy()

labels = ["before", "after"]
markers = ["o", "x"]
    
for bcd_list in [analyser.anubis_data[:max_event]]:
    cycle = 0
    points = []
    for i, bcd in enumerate(bcd_list):
        if i == 0:
            plt.scatter([bcd.hitTime], [cycle], color = "red" if bcd.error else "green")
            continue
        
        if bcd.hitTime - bcd_list[i-1].hitTime < -1e5:
            cycle += 1
        print(bcd)
        print(cycle)
        plt.scatter([bcd.hitTime], [cycle], color = "red" if bcd.error else "green")

analyser.correctionBCR()

print("After")
for bcd_list in [analyser.anubis_data[:max_event]]:
    cycle = 0
    points = []
    for i, bcd in enumerate(bcd_list):
        print(bcd)
        if i == 0:
            plt.scatter([bcd.hitTime], [cycle], color = "red" if bcd.error else "green", marker="x")
            continue
        if bcd.hitTime - bcd_list[i-1].hitTime < -1e5:
            cycle += 1
        print(cycle)
        plt.scatter([bcd.hitTime], [cycle], color = "red" if bcd.error else "green", marker="x")
    # Create a scatter plot

"""
for i in range(len(bcd_list)):
    points.append(((bcd_list[9].hitTime+114048*i) % 1048575, 2+(bcd_list[9].hitTime+114048*i)//1048575))

x_values = [point[0] for point in points]
y_values = [point[1] for point in points]


        # Create a scatter plot
plt.scatter(x_values, y_values, marker=".", label = "theory")
"""

#analyser.printBCR(10)
# Optionally, add lines connecting the points
for i in range(0,cycle,2):
    plt.axhline(y=i, color='gray', linestyle='--')
    plt.axhline(y=i+1, color='brown', linestyle='--')        
# Add labels and title
plt.xlabel('time counter')
plt.xlim(right = 1048575)
plt.ylabel('cycle')
plt.legend()
plt.title('Plot of Points')
plt.show()


BCR(997150*25/32 ns, 2024-06-27 17:23:08.002337, 2024-06-27 17:23:08.002159,2, True) 
    Triggers: [])
0
BCR(963103*25/32 ns, 2024-06-27 17:23:08.002426, 2024-06-27 17:23:08.002159,3, True) 
    Triggers: [])
0
BCR(142622*25/32 ns, 2024-06-27 17:23:08.002516, 2024-06-27 17:23:08.002159,4, True) 
    Triggers: [])
1
BCR(208158*25/32 ns, 2024-06-27 17:23:08.002605, 2024-06-27 17:23:08.002159,5, True) 
    Triggers: ['Trigger(300315*25/32 ns, 1719505388.0026767, 2879.90625)', 'Trigger(365851*25/32 ns, 1719505388.002728, 4927.90625)'])
1
BCR(370718*25/32 ns, 2024-06-27 17:23:08.002694, 2024-06-27 17:23:08.002159,6, True) 
    Triggers: [])
1
BCR(518174*25/32 ns, 2024-06-27 17:23:08.002783, 2024-06-27 17:23:08.002159,7, True) 
    Triggers: ['Trigger(498692*25/32 ns, 1719505388.0035868, 32159.15625)', 'Trigger(623107*25/32 ns, 1719505388.0028648, 3279.15625)'])
1
BCR(826910*25/32 ns, 2024-06-27 17:23:08.002872, 2024-06-27 17:23:08.002159,8, True) 
    Triggers: [])
1
BCR(974622*25/32 ns, 2

C:\Users\jony\AppData\Local\Temp\ipykernel_16448\182123382.py:67: UserWarning: No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
  plt.legend()
